In [ ]:
import sys
import os
import torch

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='1'

import numpy as np
import plotly.graph_objects as go

In [ ]:
def toDisplay(x, target_dim = 2):
    while(x.dim() > target_dim):
        x = x[0]
    return x.detach().cpu().numpy()

In [ ]:
import sys
sys.path.insert(1, '/home/exx/Documents/equivariant_pose_graph/python')
from equivariant_pose_graph.models.transformer_flow import ResidualFlow, ResidualFlow_V1
from equivariant_pose_graph.training.flow_equivariance_training_module import EquivarianceTrainingModule

# from ndf_robot.eval.test_trained_model_place import load_data
from pytorch3d.ops import sample_farthest_points
from pathlib import Path
import torch.nn.functional as F
from equivariant_pose_graph.utils.se3 import random_se3


In [ ]:
from equivariant_pose_graph.models.transformer_flow import ResidualFlow_DiffEmbTransformer
from equivariant_pose_graph.training.flow_equivariance_training_module_nocentering_eval_init import EquivarianceTestingModule
checkpoint_file='/home/exx/media/DataDrive/singularity_chuerp/epg_results/residual_flow/residual_flow_occlusion_gm/2022-06-12_015823/residual_flow_occlusion_gm/2edtw8op/checkpoints/epoch=106-step=13375.ckpt'

network = ResidualFlow_DiffEmbTransformer(
                    emb_nn='dgcnn', return_flow_component=False, center_feature=True,
                    inital_sampling_ratio=1)
model = EquivarianceTestingModule(
    network,
    lr=1e-4,
    image_log_period=100,
    weight_normalize='softmax',
    softmax_temperature = 0.1,
    loop=1
)

model.cuda()
model.load_state_dict(torch.load(checkpoint_file)['state_dict'])




In [ ]:
place_checkpoint_file='/home/exx/media/DataDrive/singularity_chuerp/epg_results/residual_flow/residual_flow_occlusion_mr/2022-06-12_024313/residual_flow_occlusion_mr/27xpsv9f/checkpoints/epoch=39-step=5000.ckpt'
place_network = ResidualFlow_DiffEmbTransformer(
                    emb_nn='dgcnn', return_flow_component=False, center_feature=True,
                    inital_sampling_ratio=1)
place_model = EquivarianceTestingModule(
    place_network,
    lr=1e-4,
    image_log_period=100,
    weight_normalize='l1',
    loop=1
)

place_model.cuda()
place_model.load_state_dict(torch.load(place_checkpoint_file)['state_dict'])

In [ ]:
def load_data(num_points, point_data, action_class, anchor_class):
    clouds = point_data['clouds'] 
    classes = point_data['classes']
    print("clouds.shape:{}".format(clouds.shape))
    print("classes.shape:{}".format(classes.shape))
    points_raw_np = clouds
    classes_raw_np = classes

    points_action_np = points_raw_np[classes_raw_np == action_class].copy()
#     points_action_mean_np = points_action_np.mean(axis=0)
#     points_action_np = points_action_np - points_action_mean_np
    
    points_anchor_np = points_raw_np[classes_raw_np == anchor_class].copy()
#     points_anchor_np = points_anchor_np - points_action_mean_np

    points_action = torch.from_numpy(points_action_np).float().unsqueeze(0)
    points_anchor = torch.from_numpy(points_anchor_np).float().unsqueeze(0)
    points_action, points_anchor = subsample(num_points,points_action, points_anchor)
    return points_action.cuda(), points_anchor.cuda()

def load_data2(num_points, point_data, action_class, anchor_class):
    clouds = point_data['clouds'] 
    classes = point_data['classes']
    pred_T_action_transformed = point_data["pred_T_action_transformed"]
    points_raw_np = clouds
    classes_raw_np = classes

    points_action_np = points_raw_np[classes_raw_np == action_class].copy()
    points_action_mean_np = points_action_np.mean(axis=0)
    pred_T_action_transformed = pred_T_action_transformed+ points_action_mean_np
    print("points_action_mean_np.shape:{}".format(points_action_mean_np.shape))
    
    points_anchor_np = points_raw_np[classes_raw_np == anchor_class].copy()
#     points_anchor_np = points_anchor_np - points_action_mean_np

    points_action = torch.from_numpy(points_action_np).float().unsqueeze(0)
    points_anchor = torch.from_numpy(points_anchor_np).float().unsqueeze(0)
    points_action, points_anchor = subsample(num_points,points_action, points_anchor)
    return torch.from_numpy(pred_T_action_transformed).cuda()

def subsample(num_points,points_action,points_anchor):
    if(points_action.shape[1] > num_points):
        points_action, _ = sample_farthest_points(points_action, 
            K=num_points, random_start_point=True)
    elif(points_action.shape[1] < num_points):
        raise NotImplementedError(f'Action point cloud is smaller than cloud size ({points_action.shape[1]} < {num_points})')

    if(points_anchor.shape[1] > num_points):
        points_anchor, _ = sample_farthest_points(points_anchor, 
            K=num_points, random_start_point=True)
    elif(points_anchor.shape[1] < num_points):
        raise NotImplementedError(f'Anchor point cloud is smaller than cloud size ({points_anchor.shape[1]} < {num_points})')
    
    return points_action, points_anchor

In [ ]:
def plot(points_action, points_anchor):
    colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]
    skip = 1
    points_action_dp = toDisplay(points_action)
    points_anchor_dp = toDisplay(points_anchor)
    go_data=[
        go.Scatter3d(x=points_action_dp[::skip,0], y=points_action_dp[::skip,1], z=points_action_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[0],
                     symbol='circle')),
        go.Scatter3d(x=points_anchor_dp[::skip,0], y=points_anchor_dp[::skip,1], z=points_anchor_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[1],
                     symbol='circle')),
    ]
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )

    fig = go.Figure(data=go_data, layout=layout)
    fig.show()

def plot_multi(plist):
    colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#e377c2',  # raspberry yogurt pink
    '#8c564b',  # chestnut brown
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]
    skip = 1
    go_data = []
    for i in range(len(plist)):
        p_dp = toDisplay(plist[i])
        plot = go.Scatter3d(x=p_dp[::skip,0], y=p_dp[::skip,1], z=p_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[i],
                     symbol='circle'))
        go_data.append(plot)
 
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )

    fig = go.Figure(data=go_data, layout=layout)
    fig.show()
def xyz2homo(xyz):
    """
    xyz:shape 1,num_points, 3
    """
    num_points = xyz.shape[1]
    homo = torch.cat([xyz.squeeze(0).detach().cpu(),torch.ones(num_points,1)],dim=-1)
    return homo
def transform(T,points):
    """
    points: num_points, 4
    """
    points = torch.permute(points,(-1,-2)) # 4,1000
    apply_here= torch.from_numpy(T).cuda()@points.cuda()
    apply_here = torch.permute(apply_here, (-1, -2))
    return apply_here[:,:3]

In [ ]:
num_classes = 3
data_idx = 11
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'pre_grasp'
# data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/test_grasp_place_demo_12_new') 
data_path = Path('/home/exx/Documents/ndf_robot/bowl_train_data_ndf_cons_3/renders') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)
 
# pred_T_action_transformed_pybullet = torch.from_numpy(point_data["pred_T_action_transformed"]).cuda()
 
# points_mug_pybullet =torch.from_numpy(point_data["points_mug"]).cuda()
# points_rack_pybullet = torch.from_numpy(point_data["points_rack"]).cuda()
# points_gripper_pybullet = torch.from_numpy(point_data["points_gripper"]).cuda()
points_action_init, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 2, anchor_class= 0)
# plot_multi([points_mug_pybullet,points_rack_pybullet,points_gripper_pybullet])

plot_multi([points_action_init,points_anchor])

In [ ]:
ans_place = place_model.get_transform(points_mug_pybullet,points_rack_pybullet)
pred_T_action_place = ans_place['pred_T_action']
mug_placed = pred_T_action_place.transform_points(points_mug_pybullet)
plot_multi([points_mug_pybullet,points_rack_pybullet,mug_placed])

In [ ]:
applied_place = transform(pred_T_action_mat, xyz2homo(points_mug_pybullet))
applied_grasp = transform(np.linalg.inv(pred_T_action_mat_mug2gripper), xyz2homo(points_gripper_pybullet))

In [ ]:
plot_multi([applied_place,points_rack_pybullet])

In [ ]:
plot_multi([applied_grasp,points_mug_pybullet])

In [ ]:
stop

In [ ]:
from pathlib import Path
import torch.nn.functional as F
from equivariant_pose_graph.utils.se3 import random_se3

num_classes = 3
data_idx = 0
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'init'
data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/test_grasp_place') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)
# print(point_data['shapenet_id'])
points_action14, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 2, anchor_class= 0)
 
plot_multi([points_action14,points_anchor])

pred_T_action_transformed = load_data2(num_points=1000, point_data=point_data, action_class= 0, anchor_class= 1)
 
 
 

In [ ]:
num_classes = 3
data_idx = 0
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'teleport'
data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/heyo1') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)
# print(point_data['shapenet_id'])
points_action14, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 0, anchor_class= 1)
 
plot_multi([points_action14,points_anchor])

pred_T_action_transformed = load_data2(num_points=1000, point_data=point_data, action_class= 0, anchor_class= 1)
 
 

In [ ]:
from pathlib import Path
import torch.nn.functional as F
from equivariant_pose_graph.utils.se3 import random_se3

num_classes = 3
data_idx = 0
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'teleport'
data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/newtest5') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)
# print(point_data['shapenet_id'])
points_action5, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 0, anchor_class= 1)
pred_T_action_transformed = load_data2(num_points=1000, point_data=point_data, action_class= 0, anchor_class= 1)
 
# pred_T_action_transformed = torch.from_numpy(point_data["pred_T_action_transformed"]).cuda()-point_data["pred_T_action_transformed"]
# print(pred_T_action_transformed.shape)
# pred_T_action_transformed[:,:,[0,1]] = pred_T_action_transformed[:,:,[1,0]]
plot_multi([points_action5,points_anchor,pred_T_action_transformed])
 

In [ ]:
num_classes = 3
data_idx = 0
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'init'
data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/newtest3') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)
pred_T_action_transformed_pybullet = torch.from_numpy(point_data["pred_T_action_transformed"]).cuda()
pred_T_action_mat = point_data["pred_T_action_mat"]
points_action_pybullet =torch.from_numpy(point_data["points_action"]).cuda()
points_anchor_pybullet = torch.from_numpy(point_data["points_anchor"]).cuda()
points_action_init, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 0, anchor_class= 1)
plot_multi([pred_T_action_transformed_pybullet,points_anchor_pybullet,points_action_pybullet])

In [ ]:
num_classes = 3
data_idx = 0
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'teleport'
data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/newtest3') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)
points_action_action3, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 0, anchor_class= 1)
# points_action_init[:,:,1]*=-1
# points_anchor[:,:,1]*=-1
# points_action_init[:,:,[1,2]]=points_action_init[:,:,[2,1]]
# points_anchor[:,:,[1,2]]=points_anchor[:,:,[2,1]]
# print(pred_T_action_mat)
# pred_T_action_mat[[1,2]] = pred_T_action_mat[[2,1]]
# print(pred_T_action_mat)
# pred_T_action_mat[:3,-1]=pred_T_action_mat[:3,:3].T@pred_T_action_mat[:3,-1]+ pred_T_action_mat[:3,-1]
# print(pred_T_action_mat)

apply_here_xyz = transform(pred_T_action_mat, xyz2homo(points_action_init))
plot_multi([apply_here_xyz,points_anchor, points_action_init,points_action_action3])

In [ ]:
ans = model.get_transform(
    points_action_init, points_anchor)  # 1, 4, 4
pred_T_action_init = ans["pred_T_action"]

In [ ]:
pred_T_action_transformed = pred_T_action_init.transform_points(
    points_action_init)
plot_multi([pred_T_action_transformed,points_anchor, points_action_init])

In [ ]:
pred_T_action_mat = pred_T_action_init.get_matrix()[0].T.detach().cpu().numpy()
apply_here_xyz = transform(pred_T_action_mat, xyz2homo(points_action_init))
plot_multi([apply_here_xyz,points_anchor, points_action_init])

In [ ]:
plot_multi([points_action0,points_anchor,points_action1,points_action3,points_action4,points_action5, points_action7])

In [ ]:
plot_multi([points_action3,points_anchor,points_action4,points_action5])

In [ ]:
stop

In [ ]:
plot(points_action, points_anchor)

In [ ]:
plot(pred_T_action_points_action, points_anchor)

In [ ]:
# Load Teleport
from pathlib import Path
import torch.nn.functional as F
from equivariant_pose_graph.utils.se3 import random_se3

num_classes = 3
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'teleport'
# data_path = Path('/home/bokorn/src/ndf_robot/notebooks')
# point_data = np.load(data_path / f'{data_idx}_obj_points.npz')
# data_path = Path('/home/exx/Documents/ndf_robot/train_new_data_3/renders')
# data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/debug_place_shapeid_34ae0b61b0d8aaf2d7b20fded0142d7a') 
data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/place_test_0_my_model_may10_overfit') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)
print(point_data['shapenet_id'])
points_action, points_anchor = load_data(num_points=1024, clouds = point_data['clouds'] ,classes = point_data['classes'], action_class= 0, anchor_class= 1)
 
points_action_trans = points_action
points_anchor_trans = points_anchor
pred_T_action, pred_T_anchor, pred_T  = place_model(points_action_trans, points_anchor_trans)
 
pred_T_action_points_action = pred_T_action.transform_points(points_action_trans)
pred_T_action_points_anchor = pred_T_anchor.transform_points(points_anchor_trans)
plot(points_action, points_anchor)
 
for i in range(20):
    pred_T_action, pred_T_anchor, pred_T  = place_model(pred_T_action_points_action, points_anchor)
    pred_T_action_points_action = pred_T_action.transform_points(pred_T_action_points_action)
    
    torch.cuda.empty_cache() 
plot(pred_T_action_points_action, points_anchor)